Election de 2022 premier tour

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import missingno as msno
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

import os
os.listdir()

['migrations',
 'election2017.ipynb',
 'pop-2007.ipynb',
 '.DS_Store',
 'populations',
 'population_2007_2023.xlsx',
 'alembic.ini',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'scripts',
 'db',
 'chomages',
 'labo2.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'elections']

#### Importer la database

In [2]:
from db import SessionLocal
session = SessionLocal()

Import des datasets.

In [3]:
tour = 2
df = pd.read_excel(f'./elections/presidentielles-2012-{tour}.xlsx')

Visualise toutes les colones

In [4]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs et nuls,% BlNuls/Ins,% BlNuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1
0,1,AIN,393866,67279.0,17.08,326587.0,82.92,19513.0,4.95,5.97,307074.0,77.96,94.03,M,HOLLANDE,François,131333.0,33.34,42.77,M,SARKOZY,Nicolas,175741.0,44.62,57.23
1,2,AISNE,376073,73997.0,19.68,302076.0,80.32,21056.0,5.60,6.97,281020.0,74.72,93.03,M,HOLLANDE,François,147260.0,39.16,52.40,M,SARKOZY,Nicolas,133760.0,35.57,47.60
2,3,ALLIER,256211,45079.0,17.59,211132.0,82.41,14924.0,5.82,7.07,196208.0,76.58,92.93,M,HOLLANDE,François,111615.0,43.56,56.89,M,SARKOZY,Nicolas,84593.0,33.02,43.11
3,4,ALPES DE HAUTE PROVENCE,123895,20314.0,16.40,103581.0,83.60,6639.0,5.36,6.41,96942.0,78.25,93.59,M,HOLLANDE,François,49498.0,39.95,51.06,M,SARKOZY,Nicolas,47444.0,38.29,48.94
4,5,HAUTES ALPES,106875,17470.0,16.35,89405.0,83.65,6025.0,5.64,6.74,83380.0,78.02,93.26,M,HOLLANDE,François,42435.0,39.71,50.89,M,SARKOZY,Nicolas,40945.0,38.31,49.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,ZS,SAINT PIERRE ET MIQUELON,4922,1531.0,31.11,3391.0,68.89,206.0,4.19,6.07,3185.0,64.71,93.93,M,HOLLANDE,François,2080.0,42.26,65.31,M,SARKOZY,Nicolas,1105.0,22.45,34.69
104,ZW,WALLIS-ET-FUTUNA,8942,2104.0,23.53,6838.0,76.47,69.0,0.77,1.01,6769.0,75.70,98.99,M,HOLLANDE,François,3795.0,42.44,56.06,M,SARKOZY,Nicolas,2974.0,33.26,43.94
105,ZX,SAINT-MARTIN/SAINT-BARTHELEMY,22686,12779.0,56.33,9907.0,43.67,415.0,1.83,4.19,9492.0,41.84,95.81,M,HOLLANDE,François,3851.0,16.98,40.57,M,SARKOZY,Nicolas,5641.0,24.87,59.43
106,ZZ,FRANCAIS DE L'ETRANGER,1078579,623669.0,57.82,454910.0,42.18,9748.0,0.90,2.14,445162.0,41.27,97.86,M,HOLLANDE,François,209002.0,19.38,46.95,M,SARKOZY,Nicolas,236160.0,21.90,53.05


In [5]:
#df.drop(index=107, inplace=True)

supprimer la colonne Etat saisie d’un DataFrame pandas.

In [6]:
df = df.drop(columns=["Etat saisie"], errors="ignore")

In [7]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df.head(1000)

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs et nuls,% BlNuls/Ins,% BlNuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Sexe.1,Nom.1,Prénom.1,Voix.1,% Voix/Ins.1,% Voix/Exp.1
0,1,AIN,393866,67279.0,17.08,326587.0,82.92,19513.0,4.95,5.97,307074.0,77.96,94.03,M,HOLLANDE,François,131333.0,33.34,42.77,M,SARKOZY,Nicolas,175741.0,44.62,57.23
1,2,AISNE,376073,73997.0,19.68,302076.0,80.32,21056.0,5.60,6.97,281020.0,74.72,93.03,M,HOLLANDE,François,147260.0,39.16,52.40,M,SARKOZY,Nicolas,133760.0,35.57,47.60
2,3,ALLIER,256211,45079.0,17.59,211132.0,82.41,14924.0,5.82,7.07,196208.0,76.58,92.93,M,HOLLANDE,François,111615.0,43.56,56.89,M,SARKOZY,Nicolas,84593.0,33.02,43.11
3,4,ALPES DE HAUTE PROVENCE,123895,20314.0,16.40,103581.0,83.60,6639.0,5.36,6.41,96942.0,78.25,93.59,M,HOLLANDE,François,49498.0,39.95,51.06,M,SARKOZY,Nicolas,47444.0,38.29,48.94
4,5,HAUTES ALPES,106875,17470.0,16.35,89405.0,83.65,6025.0,5.64,6.74,83380.0,78.02,93.26,M,HOLLANDE,François,42435.0,39.71,50.89,M,SARKOZY,Nicolas,40945.0,38.31,49.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,ZS,SAINT PIERRE ET MIQUELON,4922,1531.0,31.11,3391.0,68.89,206.0,4.19,6.07,3185.0,64.71,93.93,M,HOLLANDE,François,2080.0,42.26,65.31,M,SARKOZY,Nicolas,1105.0,22.45,34.69
104,ZW,WALLIS-ET-FUTUNA,8942,2104.0,23.53,6838.0,76.47,69.0,0.77,1.01,6769.0,75.70,98.99,M,HOLLANDE,François,3795.0,42.44,56.06,M,SARKOZY,Nicolas,2974.0,33.26,43.94
105,ZX,SAINT-MARTIN/SAINT-BARTHELEMY,22686,12779.0,56.33,9907.0,43.67,415.0,1.83,4.19,9492.0,41.84,95.81,M,HOLLANDE,François,3851.0,16.98,40.57,M,SARKOZY,Nicolas,5641.0,24.87,59.43
106,ZZ,FRANCAIS DE L'ETRANGER,1078579,623669.0,57.82,454910.0,42.18,9748.0,0.90,2.14,445162.0,41.27,97.86,M,HOLLANDE,François,209002.0,19.38,46.95,M,SARKOZY,Nicolas,236160.0,21.90,53.05


In [8]:
df.shape

(108, 25)

Recuperer les information généraux sur les élections départemental

In [9]:
df_infos_general = df.iloc[:, :13]
df_infos_general.head()

,Code du département,Libellé du département,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs et nuls,% BlNuls/Ins,% BlNuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot
0,1,AIN,393866,67279.0,17.08,326587.0,82.92,19513.0,4.95,5.97,307074.0,77.96,94.03
1,2,AISNE,376073,73997.0,19.68,302076.0,80.32,21056.0,5.60,6.97,281020.0,74.72,93.03
2,3,ALLIER,256211,45079.0,17.59,211132.0,82.41,14924.0,5.82,7.07,196208.0,76.58,92.93
3,4,ALPES DE HAUTE PROVENCE,123895,20314.0,16.40,103581.0,83.60,6639.0,5.36,6.41,96942.0,78.25,93.59
4,5,HAUTES ALPES,106875,17470.0,16.35,89405.0,83.65,6025.0,5.64,6.74,83380.0,78.02,93.26


Renommer les colonnes des information généraux sur les élections départemental

In [10]:
df_infos_general.columns = [
    "code_dept", "nom_dept", "nb_inscrits", "nb_abstentions",
    "pct_abstentions", "nb_votants", "pct_votants", "nb_blancs_nuls",
    "pct_blancs_nuls_inscrits", "pct_blancs_nuls_votants", "nb_exprimes",
    "pct_exprimes_inscrits", "pct_exprimes_votants"
]
df_infos_general.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,pct_abstentions,nb_votants,pct_votants,nb_blancs_nuls,pct_blancs_nuls_inscrits,pct_blancs_nuls_votants,nb_exprimes,pct_exprimes_inscrits,pct_exprimes_votants
0,1,AIN,393866,67279.0,17.08,326587.0,82.92,19513.0,4.95,5.97,307074.0,77.96,94.03
1,2,AISNE,376073,73997.0,19.68,302076.0,80.32,21056.0,5.60,6.97,281020.0,74.72,93.03
2,3,ALLIER,256211,45079.0,17.59,211132.0,82.41,14924.0,5.82,7.07,196208.0,76.58,92.93
3,4,ALPES DE HAUTE PROVENCE,123895,20314.0,16.40,103581.0,83.60,6639.0,5.36,6.41,96942.0,78.25,93.59
4,5,HAUTES ALPES,106875,17470.0,16.35,89405.0,83.65,6025.0,5.64,6.74,83380.0,78.02,93.26


In [11]:
df_infos_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   code_dept                 107 non-null    object 
 1   nom_dept                  107 non-null    object 
 2   nb_inscrits               108 non-null    int64  
 3   nb_abstentions            107 non-null    float64
 4   pct_abstentions           107 non-null    float64
 5   nb_votants                107 non-null    float64
 6   pct_votants               107 non-null    float64
 7   nb_blancs_nuls            107 non-null    float64
 8   pct_blancs_nuls_inscrits  107 non-null    float64
 9   pct_blancs_nuls_votants   107 non-null    float64
 10  nb_exprimes               107 non-null    float64
 11  pct_exprimes_inscrits     107 non-null    float64
 12  pct_exprimes_votants      107 non-null    float64
dtypes: float64(10), int64(1), object(2)
memory usage: 11.1+ KB


Suppression des données redondantes, risque de multicolinéarité en ML

In [12]:
colonnes_base = [
    "code_dept", "nom_dept",
    "nb_inscrits", "nb_abstentions", "nb_votants", 
    "nb_blancs_nuls",
]

df_infos_general_base = df_infos_general[colonnes_base]
df_infos_general_base.head()

,code_dept,nom_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,AIN,393866,67279.0,326587.0,19513.0
1,2,AISNE,376073,73997.0,302076.0,21056.0
2,3,ALLIER,256211,45079.0,211132.0,14924.0
3,4,ALPES DE HAUTE PROVENCE,123895,20314.0,103581.0,6639.0
4,5,HAUTES ALPES,106875,17470.0,89405.0,6025.0


In [13]:
df_infos_general_base.shape

(108, 6)

In [14]:
df_departement = df_infos_general_base.iloc[:, :2]

pd.set_option('display.max_rows', None)      # Affiche toutes les lignes
pd.set_option('display.max_columns', None)   # Affiche toutes les colonnes
pd.set_option('display.width', None)         # Évite le retour à la ligne automatique
pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des cellules

print(df_departement)
#df_departement.head()

    code_dept                       nom_dept
0           1                            AIN
1           2                          AISNE
2           3                         ALLIER
3           4        ALPES DE HAUTE PROVENCE
4           5                   HAUTES ALPES
5           6                ALPES MARITIMES
6           7                        ARDECHE
7           8                       ARDENNES
8           9                         ARIEGE
9          10                           AUBE
10         11                           AUDE
11         12                        AVEYRON
12         13               BOUCHES DU RHONE
13         14                       CALVADOS
14         15                         CANTAL
15         16                       CHARENTE
16         17              CHARENTE MARITIME
17         18                           CHER
18         19                        CORREZE
19         2A                      CORSE SUD
20         2B                    HAUTE CORSE
21        

In [15]:
df_stat_elections = df_infos_general_base.drop(columns=["nom_dept"])
df_stat_elections.head()

,code_dept,nb_inscrits,nb_abstentions,nb_votants,nb_blancs_nuls
0,1,393866,67279.0,326587.0,19513.0
1,2,376073,73997.0,302076.0,21056.0
2,3,256211,45079.0,211132.0,14924.0
3,4,123895,20314.0,103581.0,6639.0
4,5,106875,17470.0,89405.0,6025.0


Tratification pour consever les proportions

In [16]:
import pandas as pd

# Colonnes globales que l'on veut garder
cols_globales = ['Code du département', 'Libellé du département']

# Colonnes répétitives candidats
df_candidates = df.iloc[:, 13:]  # après % Exp/Vot
cols_per_candidate = 6
num_candidates = df_candidates.shape[1] // cols_per_candidate

dfs_list = []

for i in range(num_candidates):
    start_col = i * cols_per_candidate
    end_col = start_col + cols_per_candidate
    df_cand = df_candidates.iloc[:, start_col:end_col].copy()
    df_cand.columns = ['Sexe', 'Nom', 'Prenom', 'Voix', '% Voix/Ins', '% Voix/Exp']
    
    # Ajouter les informations de département
    df_cand = pd.concat([df[cols_globales], df_cand], axis=1)
    
    dfs_list.append(df_cand)

# Combiner tous les candidats
df_simple = pd.concat(dfs_list, ignore_index=True)

# Colonnes à garder
cols_to_keep = ['Code du département', 'Libellé du département', 'Sexe', 'Nom', 'Prenom', 'Voix']

# Nouveau dataframe simplifié
df_candidat_resultat = df_simple[cols_to_keep]

# Afficher toutes les lignes
pd.set_option('display.max_rows', 10)  # ou un nombre supérieur à 100
pd.set_option('display.max_columns', None)  # pour voir toutes les colonnes
pd.set_option('display.width', None)        # pour éviter la coupure à la largeur de l'écran



#df_candidat_resultat = df_candidat_resultat.drop(index=215)
# Affichage des 5 premières lignes
df_candidat_resultat.head(1000)


,Code du département,Libellé du département,Sexe,Nom,Prenom,Voix
0,1,AIN,M,HOLLANDE,François,131333.0
1,2,AISNE,M,HOLLANDE,François,147260.0
2,3,ALLIER,M,HOLLANDE,François,111615.0
3,4,ALPES DE HAUTE PROVENCE,M,HOLLANDE,François,49498.0
4,5,HAUTES ALPES,M,HOLLANDE,François,42435.0
...,...,...,...,...,...,...
211,ZS,SAINT PIERRE ET MIQUELON,M,SARKOZY,Nicolas,1105.0
212,ZW,WALLIS-ET-FUTUNA,M,SARKOZY,Nicolas,2974.0
213,ZX,SAINT-MARTIN/SAINT-BARTHELEMY,M,SARKOZY,Nicolas,5641.0
214,ZZ,FRANCAIS DE L'ETRANGER,M,SARKOZY,Nicolas,236160.0


In [17]:
from datetime import date

from utils.election_importer import ElectionImporter
from enums.type_election import TypeElection

# =====================================================
# Instanciation
# =====================================================
importer = ElectionImporter()

# =====================================================
# Élection
# =====================================================
if tour == 1:
    election = importer.get_or_create_election(
        election_date=date(2012, 4, 22),  # 1er tour présidentielle 2012
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

elif tour == 2:
    election = importer.get_or_create_election(
        election_date=date(2012, 5, 6),  # 2nd tour présidentielle 2012
        type_election=TypeElection.PRESIDENTIELLE,
        tour=tour
    )

# =====================================================
# Départements
# =====================================================
#df_departement = edf.df_departement
if df_departement is not None and not df_departement.empty:
    importer.import_departements(df_departement)

# =====================================================
# Statistiques par département
# =====================================================
#df_stat_elections = edf.df_stat_elections
if df_stat_elections is not None and not df_stat_elections.empty:
    importer.import_stats(df_stat_elections, election.id)

# =====================================================
# Candidats & résultats
# =====================================================
#df_candidat_resultat = edf.df_candidat_resultat
if df_candidat_resultat is not None and not df_candidat_resultat.empty:
    importer.import_candidats_resultats(df_candidat_resultat, election.id)

print("✅ Import élection 2017 – Tour 1 terminé avec succès")


2026-01-04 07:41:48,497 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2026-01-04 07:41:48,498 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-04 07:41:48,501 INFO sqlalchemy.engine.Engine select current_schema()
2026-01-04 07:41:48,501 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-04 07:41:48,503 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2026-01-04 07:41:48,503 INFO sqlalchemy.engine.Engine [raw sql] {}
2026-01-04 07:41:48,505 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-01-04 07:41:48,508 INFO sqlalchemy.engine.Engine SELECT elections.id AS elections_id, elections.date AS elections_date, elections.type_election AS elections_type_election, elections.tour AS elections_tour 
FROM elections 
WHERE elections.date = %(date_1)s AND elections.type_election = %(type_election_1)s AND elections.tour = %(tour_1)s 
 LIMIT %(param_1)s
2026-01-04 07:41:48,509 INFO sqlalchemy.engine.Engine [generated in 0.00092s] {'date_1': datetime.date(2012, 5, 6), 't

AttributeError: 'float' object has no attribute 'lower'